In [31]:
import os
import pandas as pd
import numpy as np
import re

In [32]:
block_files_labelled = {
    0 : "Constituency_Name",
    1 : "Candidates_Info",
    2 : "Electors_Info",
    3 : "Voters_Info",
    4 : "Votes_Info",
    5 : "Polling_Stations",
    6 : "Dates",
    7 : "Result"
}

In [33]:
def x_drop_empty_columns(df):
    n_df = df.copy()
    empty_cols = [(col,empty) for col,empty in n_df.isna().all().items()]
    for (n_col, n_empty) in empty_cols:
        if(n_col in ['MALE', 'FEMALE', 'TOTAL']):
            continue

        if(n_empty):
            n_df = n_df.drop(n_col, axis = 'columns')
            
    return n_df


def x_drop_empty_rows(df):
    n_df = df.copy()
    empty_rows = n_df.isna().all(axis = 'columns')
    n_df = n_df[~empty_rows].copy()
    return n_df

In [34]:
block_size = 8
TARGET_STATE = "Maharashtra"

In [35]:
ALL_YEARS = [int(i) for i in os.listdir(os.path.join('states', TARGET_STATE)) if(not i.startswith("."))]

In [36]:
for year in sorted(ALL_YEARS):
    print(year)

1962
1967
1972
1978
1980
1985
1990
1995
1999
2004
2009
2014


In [37]:
TARGET_YEAR = 2014

In [38]:
# tar_dir = os.path.join('states', 'states', TARGET_STATE, f"{TARGET_YEAR}", f"tabula-{TARGET_YEAR}_constituency_summary")
tar_dir = os.path.join('states', TARGET_STATE, f"{TARGET_YEAR}", f"tabula-{TARGET_YEAR}_ConstituencyData")
csv_files = [i for i in os.listdir(tar_dir) if(i.endswith("csv"))]
file_df = pd.DataFrame({"file" : csv_files})
file_df['file_num'] = file_df['file'].str.split('-').str[-1].str.split(".").str[0].astype("int")
file_df = file_df.sort_values('file_num')

In [39]:
from collections import Counter

In [46]:
constituency_wise_data = {}
completeness_ratios = {}

for i in range(0, len(csv_files), block_size):
    tar_files = file_df['file'].to_list()[i : i+block_size]
    tmp_dfs = {idx : os.path.join(tar_dir, file) for idx,file in enumerate(tar_files)}
    
    const_found = False
    current_constituency = 0
    
    # Men, Women, Others
    n_genders = 2
    uncontested = False
    
    for idx, df_path in tmp_dfs.items():
        header_match = {
            7 : [0,1],
            5 : None
        }
        header = header_match.get(idx, [0])
        df = pd.read_csv(df_path, header = header)
        if(idx == 5):
            # drop first row
            df = df.iloc[1:, :]
        
        if(idx == 0):
            tar_val = ' '.join(df.columns)
            tar_obj = re.match(".+ (\d+)\s*-\s*([A-Za-z ]+)", tar_val)
            if(tar_obj):
                const_found = True
                const_num = int(tar_obj[1])
                current_constituency = const_num
                constituency_wise_data[const_num] = {}
                constituency_wise_data[current_constituency]['Constituency_Name'] = tar_obj[2]
                completeness_ratios[current_constituency] = {}
                
            else:
                print(tar_files)
                display(df)
                raise ValueError("Could not extract constituency number")
                
                
            
        else:
            n_df = df.copy()
            
            if(idx == 4 and uncontested):
                n_df['VALUE'] = -1
            
            # Polling Dates
            if(idx == 6):
                n_df.columns = n_df.iloc[0]
                n_df = n_df.drop([0], axis = 'index')
                n_df.columns = n_df.columns.fillna("k")
                
            # idx1,2,3 have some partially empty first cols... let's merge these and drop the extras
            if(idx == 1):
                tar_cols = [i for i in n_df.columns if(i.startswith("Unnamed"))]
                if(len(tar_cols) > 1):
                    keep_col = tar_cols[0]
                    n_df[keep_col] = n_df[tar_cols].fillna("").agg(lambda x: ' '.join(x), axis = 'columns')
                    n_df = n_df.drop(tar_cols[1:], axis = 'columns')
                
            # Drop empty columns, if any
            n_df = x_drop_empty_columns(n_df)
                    
            
            
            
            # Now parse the blocks separately...
            # Note that block 0 has already been parsed
            
            # Block 5,6,7
            if(idx in [5, 6, 7]):
                # Polling Stations
                if(idx == 5):
                    
                    # Find row that has a regex match first
                    info_found = False
                    found_obj = None
                    for b_idx, b_row in n_df.iterrows():
                        tar_row =  ' '.join([str(i) for i in b_row.dropna()])
                        tar_obj = re.match("[A-Z]+\s*:?\s*(\d+)? [A-Z ]+:?\s*(\d+)", tar_row)
                        if(tar_obj):
                            info_found = True
                            found_obj = tar_obj
                            break

                    if(info_found):
                        num_stations = found_obj[1]
                        avg_electors = found_obj[2]
                        
                    else:
                        num_stations = -1
                        avg_electors = -1
                        
                    constituency_wise_data[const_num]['Polling_Stations'] = {
                        "Number" : num_stations,
                        "Average_Electors_per_Station" : avg_electors,
                        "regex_Match" : info_found
                    }
                
                # Dates
                elif(idx == 6):
                    if(uncontested or n_df.empty):
                        n_df = pd.DataFrame({"POLLING" : -1}, index = [1])
                    
                    n_df.columns = [f"DATE_{i}" for i in n_df.columns]
                    n_df = x_drop_empty_rows(n_df)
                    n_df = n_df.reset_index(drop = True)
                    constituency_wise_data[const_num].update({
                        "Dates" : n_df.T.to_dict()[0]
                    })

                    
                
                # Candidates Info
                elif(idx == 7):
                    n_df = x_drop_empty_rows(n_df)
                    
                    # Manually deal with uncontested
                    if(uncontested):
                        # Multi-indexed column
                        n_df.columns = [i[1] for i in n_df.columns]
                        pick_rows = ['PARTY', 'CANDIDATE', 'VOTES']
                        n_df = n_df[pick_rows]
                        n_df = n_df.dropna(how = 'any')
                        n_df['POSITION'] = 'WINNER'
                        n_df['VOTES'] = -1
                        if(n_df['CANDIDATE'].str.contains('UNCONTESTED').any()):
                            n_df['CANDIDATE'] = n_df['CANDIDATE'].str.replace("UNCONTESTED", "")
                            n_df['CANDIDATE'] = n_df['CANDIDATE'].str.removesuffix("(").str.strip()
                    
                    else:
                        n_df.columns = ['POSITION', 'PARTY', 'CANDIDATE', 'VOTES']
                        n_df['POSITION'] = n_df['POSITION'].str.removesuffix(":").str.strip().str.upper().str.replace(" ", "-")
                    
                    na_values_count = n_df.isna().sum().sum()
                    total_values = n_df.shape[0] * n_df.shape[1]
                    completeness_ratios[current_constituency]['Result'] = {
                        "Total_Cells" : total_values,
                        "NA_Cells" : na_values_count,
                        "NA_Proportion" : (na_values_count / total_values) * 100
                    }
                    
                    # Multi-line names - merge with previous line
                    if(len(n_df) > 2):
                        print("[FOUND] Multi-line name problem... Fixing with iterative row merge")
                        display(n_df)
                        for row_idx, row_vals in n_df.iterrows():
                            if(row_vals.isna().any()):
                                na_cols = ~row_vals.isna()
                                for k,v in na_cols.items():
                                    if(v):
                                        n_df.loc[row_idx - 1, k] += " " + str(n_df.loc[row_idx, k])

                                n_df = n_df.drop(row_idx, axis = 'index')
                            

                    constituency_wise_data[const_num]['Result'] = n_df.set_index('POSITION').T.to_dict()

            else:
                variable_name = n_df.columns[0]
                for col in n_df.columns:
                    if(col == variable_name):
                        continue
                        
                    if(not uncontested and (idx == 3)):
                        if("Unnamed: 1" in n_df.columns):
                            z_vals = n_df['Unnamed: 1'].dropna().values
                            if(len(z_vals) == 1 and z_vals[0].lower().strip() == "uncontested"):
                                uncontested = True
                                n_df['Unnamed: 1'] = -1
                                n_df['FEMALE'] = -1
                                if(n_genders == 3):
                                    n_df['Others'] = -1
                                n_df['TOTAL'] = -1
                    
                    constituency_wise_data[current_constituency]['Uncontested'] = uncontested            
                                
                        
                    n_df[col] = n_df[col].astype("Int64")
                    
                
                n_df['TAR_VARIABLE'] = n_df[variable_name].str.extract("\d+\.\s*([A-Z ]+)")
                if(idx == 4):
                    # Iteratively merge the rows
                    last_good_row = None
                    
                    for gz_idx, gz_row in n_df.iterrows():
                        if(not pd.isna(gz_row['TAR_VARIABLE'])):
                            last_good_row = gz_idx
                        
                        if(pd.isna(gz_row['TAR_VARIABLE']) and not pd.isna(gz_row[variable_name])):
                            
                            n_df.loc[last_good_row, 'TAR_VARIABLE'] += " " + gz_row[variable_name]
                            
                    # after merging, create a new df with the correct variable names & values
                    tmp_variable_names = n_df['TAR_VARIABLE'].dropna().to_list()
                    tar_variable_values_column = [i for i in n_df.columns if(i not in ['TAR_VARIABLE', variable_name])]
                    
                    # unable to pick the VALUE column...
                    if(len(tar_variable_values_column) > 1):
                        raise ValueError("could not find values column!")
                        
                    tar_variable_values_column = tar_variable_values_column[0]
                    tmp_variable_values = n_df[tar_variable_values_column].dropna().to_list()
                    
                    # keep variable name column only because it is kept for other dfs as well, will drop later...
                    if(len(tmp_variable_values) != len(tmp_variable_names)):
                        display(n_df)
                        print(tar_files)
                        print(tmp_variable_values)
                        print(tmp_variable_names)
                    n_df = pd.DataFrame({
                        'VALUE' : tmp_variable_values,
                        'TAR_VARIABLE' : tmp_variable_names,
                        variable_name : tmp_variable_names
                    })

                zero_rows = (n_df == 0).all(axis = 'columns') & (n_df['TAR_VARIABLE'] == 0)
                n_df = n_df.drop(zero_rows[zero_rows].index)
                
                
                n_df = n_df.drop(variable_name, axis = 'columns')
                
                # Code to shift variables
                for z_idx, z_row in n_df.iterrows():
                    if(z_row.isna().any()):
                        other_keys = [i for i in z_row.keys() if(i != "TAR_VARIABLE")]
                        keep_values = z_row[other_keys].dropna().values
                        num_of_na = z_row[other_keys].isna().sum()
                        na_values = [pd.NA for i in range(num_of_na)]
                        n_df.loc[z_idx, other_keys] = list(keep_values) + na_values
                        

                # Drop empty columns again
                n_df = x_drop_empty_columns(n_df)
                
                # Drop rows that are entirely empty as well
                n_df = x_drop_empty_rows(n_df)
                
                block_name = block_files_labelled[idx]
                na_values_count = n_df.isna().sum().sum()
                total_values = n_df.shape[0] * n_df.shape[1]
                completeness_ratios[current_constituency][block_name] = {
                    "Total_Cells" : total_values,
                    "NA_Cells" : na_values_count,
                    "NA_Proportion" : (na_values_count / total_values) * 100
                }
                    
                n_df = n_df.fillna(0)
                
                # Set n_gender correctly
                if(idx == 1):
                    n_genders = sum([True for i in n_df.columns if(i in ['MALE', 'MEN', 'WOMEN', 'FEMALE', 'Others', 'OTHERS', 'THIRD GENDER'])])
                
                
                # Set column names correctly
                if(idx == 4):
                    out_columns = ['VALUE', 'TAR_VARIABLE']
                    
                else:
                    out_columns = ['MEN', 'WOMEN', 'OTHERS', 'TOTAL', 'TAR_VARIABLE']
                    if(n_genders == 2):
                        out_columns = ['MEN', 'WOMEN', 'TOTAL', 'TAR_VARIABLE']
                        
                if(len(n_df.columns) != len(out_columns)):
                    n_df.insert(loc = 1, column = 'WOMEN', value = 0)
                    constituency_wise_data[current_constituency]['Women_Column_Forced'] = True
                    
                n_df.columns = out_columns
                
                # Keep data
                block_name = block_files_labelled[idx]
                
                gz_tar_cols = n_df.set_index('TAR_VARIABLE').T.columns
                gz_count = Counter(gz_tar_cols)
                if(gz_count.most_common(1)[0][1] > 1):
                    print(tar_files)
                    display(n_df)
                
                constituency_wise_data[current_constituency][block_name] = n_df.set_index('TAR_VARIABLE').T.to_dict()

,IV. VOTES,Unnamed: 1,TAR_VARIABLE
0,1. TOTAL VOTES POLLED ON EVM,<NA>,TOTAL VOTES POLLED ON EVM
1,2. TOTAL DEDUCTED VOTES FROM EVM(TEST,<NA>,TOTAL DEDUCTED VOTES FROM EVM VOTES+VOTES NOT ...
2,VOTES+VOTES NOT RETRIVED+VOTES REJECTED DUE TO,<NA>,NaN
3,OTHER REASONS + 'NOTA'),<NA>,NaN
4,3. TOTALVALID VOTES POLLED ON EVM,<NA>,TOTALVALID VOTES POLLED ON EVM
5,4. POSTAL VOTES COUNTED,<NA>,POSTAL VOTES COUNTED
6,NaN,511,NaN
7,5. POSTAL VOTES DEDUCTED(REJECTED POSTAL,<NA>,POSTAL VOTES DEDUCTED VOTES + POSTAL VOTES POL...
8,VOTES + POSTAL VOTES POLLED FOR 'NOTA'),<NA>,NaN
9,NaN,9,NaN


['tabula-2014_ConstituencyData-1808.csv', 'tabula-2014_ConstituencyData-1809.csv', 'tabula-2014_ConstituencyData-1810.csv', 'tabula-2014_ConstituencyData-1811.csv', 'tabula-2014_ConstituencyData-1812.csv', 'tabula-2014_ConstituencyData-1813.csv', 'tabula-2014_ConstituencyData-1814.csv', 'tabula-2014_ConstituencyData-1815.csv']
[511, 9, 502, 1362, 0]
['TOTAL VOTES POLLED ON EVM', "TOTAL DEDUCTED VOTES FROM EVM VOTES+VOTES NOT RETRIVED+VOTES REJECTED DUE TO OTHER REASONS + 'NOTA')", 'TOTALVALID VOTES POLLED ON EVM', 'POSTAL VOTES COUNTED', "POSTAL VOTES DEDUCTED VOTES + POSTAL VOTES POLLED FOR 'NOTA')", 'VALID POSTAL VOTES', 'TOTAL VALID VOTES POLLED', 'TEST VOTES POLLED ON EVM', 'VOTES POLLED FOR ', 'TENDERED VOTES']


ValueError: All arrays must be of the same length

In [ ]:
state_df = pd.DataFrame()
for const_num in constituency_wise_data.keys():
    const_df = pd.DataFrame(
        {
            "Constituency_No" : [const_num], 
             "Uncontested" : [constituency_wise_data[const_num]['Uncontested']],
             "Women_Col_Forced" : [constituency_wise_data[const_num].get('Women_Column_Forced', False)]
        }
    )
    # Dictionary^2
    tar_keys2 = ['Dates', 'Polling_Stations']
    for key in tar_keys2:
        for k,v in constituency_wise_data[const_num][key].items():
            colname = f"{key}_{k}"
            const_df[colname] = v
            
    
    # Dictionary^3
    tar_keys = ['Candidates_Info', 'Result', 'Electors_Info', 'Voters_Info', 'Votes_Info']
    for key in tar_keys:
        for k,v in constituency_wise_data[const_num][key].items():
            cols = {f"{key.strip()}_{k.strip()}_{subsection}" : value for subsection, value in v.items()}
            x_df = pd.DataFrame(cols, index = [0])
            const_df = pd.concat([const_df, x_df], axis = 1)

        completeness_df = pd.DataFrame(completeness_ratios[const_num][key], index = [0])
        completeness_df.columns = [f"Completeness_{key}_{i}" for i in completeness_df.columns]
        const_df = pd.concat([const_df, completeness_df], axis = 1)
            
    state_df = pd.concat([state_df, const_df])

In [313]:
outpath = os.path.join("Parsed_Data", TARGET_STATE)
os.makedirs(outpath, exist_ok = True)

In [314]:
out_filename = f"{TARGET_STATE}_{TARGET_YEAR}.csv"
state_df.to_csv(
    os.path.join(outpath, out_filename),
    index = False
)

In [291]:
print(state_df.head())

   Constituency_No  Uncontested  Women_Col_Forced Dates_DATE_POLLING  \
0                1        False             False         21-02-1967   
0                2        False             False         21-02-1967   
0                3        False             False         21-02-1967   
0                4        False             False         21-02-1967   
0                5        False             False         21-02-1967   

   Polling_Stations_Number  Polling_Stations_Average_Electors_per_Station  \
0                       -1                                             -1   
0                       -1                                             -1   
0                       -1                                             -1   
0                       -1                                             -1   
0                       -1                                             -1   

   Polling_Stations_regex_Match  Candidates_Info_NOMINATED_MEN  \
0                         False       